In [1]:
import os
import numpy as np
import json
import pennylane as qml
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from qutip import tensor, basis
import time

from dll.baseline_dru_v2 import Modelo_DRU as Modelo_DRU_v2
from dll.cost_functions import Renyi_Divergence_2
from dll.main_fun import make_IRIS_v2, density_matrix_1, _test_pulse_model
from dll.models import qcircuit_1_qubit_mixed, qcircuit_2_qubit_mixed, qcircuit_4_qubit_mixed
from dll.gate_procesor_modificated import CompositeGateProcessor

In [2]:
def fidelity(state0, state1):
  F  = qml.math.fidelity(state0, state1)
  return F

def _test_pulse_model_(state_output, state_labels):
  fidelities = []
  for dm in state_labels:
    f = fidelity(state_output, dm)
    fidelities.append(f)
  best_fidel = np.argmax(fidelities)
  return best_fidel

In [3]:
# call dataset
X, y =  make_IRIS_v2(3,3)

# Crear el objeto KFold para generar las divisiones
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# lista de splits
kf_splits = list(kf.split(X))

# Seleccionar pruebas
kf_splits_2 = [kf_splits[0], kf_splits[1], kf_splits[2], kf_splits[3],kf_splits[4]]

In [5]:
# Definicion de ruido
d_latice = 3 # cantidad de puntos
epocas = 10
layers = 5
N_qubits = 2
nombre_funcion = f"qcircuit_{N_qubits}_qubit_mixed"
model_circuit = globals().get(nombre_funcion)
folds_th = 1
f_loss = Renyi_Divergence_2
entan = True

# ruido decoherente
start = np.log10(0.001e-6)  # Logaritmo base 10 de 0.001
stop = np.log10(10e-6)     # Logaritmo base 10 de 0.05
# Genera 10 puntos uniformemente distribuidos en escala logarítmica
T2_range = np.logspace(start, stop, num=d_latice)
T2start_range = T2_range*5
T1_range = (1/2)*((T2_range*T2start_range)/(T2start_range - T2_range))

# ruido coherente
start = np.log10(0.0010)  # Logaritmo base 10 de 0.001
stop = np.log10(0.05)     # Logaritmo base 10 de 0.05
# Genera 10 puntos uniformemente distribuidos en escala logarítmica
range_std = np.logspace(start, stop, num=d_latice)

# Iterar sobre las divisiones generadas por KFold
AUC = []
fold_ith = 1


inicio = time.time()
for tpl_train_indextest_index in kf_splits_2[:folds_th]:
    train_index, test_index  = tpl_train_indextest_index
    Auc_ij = np.zeros((len(range_std), len(T2_range)))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = Modelo_DRU_v2(model_circuit, f_loss,
                        epochs = epocas, n_clases = 3, n_qubits = N_qubits,
                        features = 3, num_layers = layers, alpha_noise = 0.0, val_prc = 0.2,
                        save_process = False, entanglement = entan)
    paramters_1q, bias_1q = model.fit(X_train, y_train, X_test, y_test)
    for i in range(len(range_std)):
        for j in range(len(T2_range)):
            # test in DRU in spins
            y_pred = []
            # Init class:
            dru_circuit = CompositeGateProcessor(n_points_pulse_Ri=70, n_points_pulse_Ri_spl = 70, version_qutip="5",
                                                    N_qubits = N_qubits, tf_noise = True, noise_std = range_std[i],
                                                    tf_quantum_noise = True, T1=T1_range[j], T2=T2_range[j],
                                                    ket_dru_0 = tensor([basis(2, 0)]*N_qubits)
                                                )
            for x in X_test:
                metodo = getattr(dru_circuit, f"qcircuit_DRU_{N_qubits}_Qubit", None)
                state_dru = metodo(paramters_1q, x, bias=bias_1q, entanglement=entan)
                dru_circuit.reset_ket_0()
                dm_out = density_matrix_1(state_dru.full())
                clas_pred = _test_pulse_model_(dm_out, model.state_labels)
                y_pred.append(clas_pred)
                # calculos auc
            encoder = OneHotEncoder()
            y_test_one_hot = encoder.fit_transform(y_test.reshape(-1, 1))
            y_pred_one_hot = encoder.transform(np.array(y_pred).reshape(-1, 1))
            auc_score = roc_auc_score(y_test_one_hot.toarray(), y_pred_one_hot.toarray(),  multi_class='ovo')
            Auc_ij[i,j] = auc_score
    python_list = Auc_ij.tolist()
    # Guardar la lista en un archivo JSON
    with open(f'data/AUC_{N_qubits}q_{f_loss.__name__}_thfold_{fold_ith}_NL_{layers}_entang_{entan}.json', 'w') as json_file:
        json.dump(python_list, json_file)
    fold_ith += 1
    
fin = time.time()
# Calcular el tiempo transcurrido
tiempo_transcurrido = fin - inicio
print("El código tardó {:.5f} segundos en ejecutarse.".format(tiempo_transcurrido))

Epoch: 100%|██████████| 10/10 [00:37<00:00,  3.72s/it]
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312

El código tardó 287.94405 segundos en ejecutarse.
